<a href="https://colab.research.google.com/github/ttogle918/news_by_kobert/blob/master/STS/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%9D%BC%EB%B2%A8%EB%A7%81_%EC%A6%9D%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델 가져오기 시도

데이터 증식은 성공!

In [ ]:
!pip install sentencepiece transformers torch

In [ ]:
!pip install seqeval
!pip install torchtext pytorch-lightning
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
!pip install sentence_transformers numpy scikit-learn scipy nltk tqdm

In [2]:
import pytorch_lightning as pl
pl.__version__

'1.5.10'

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [4]:
from transformers import BertConfig, BertModel, AdamW

In [5]:
from tqdm import tqdm, tqdm_notebook
from typing import Callable, List, Tuple
from seqeval.metrics import f1_score, accuracy_score

In [6]:
import torch
# gpu 연산이 가능하면 'cuda:0', 아니면 'cpu' 출력
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device, torch.cuda.device_count()

(device(type='cuda', index=0), 1)

In [7]:
from google.colab import drive

drive.mount('/content/drive')
data_path = '/content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400'

Mounted at /content/drive


In [40]:
def text_splicing(text, max_len) :
  before_idx = 0
  li = []
  i = 0
  while True:
    if before_idx+max_len < len(text) :
      li.append( [text[before_idx:before_idx+max_len], i] )
      i += 1
    else :
      li.append(  [text[before_idx:], i] )
      if len(li) < 2 :
        return None   # 연결할 단락이 없으므로 dataset에 넣지 않는다.
      return li

    before_idx += max_len
  return li

In [41]:
max_len = 256

In [42]:
text = """특정한 풍경을 통하여 개인적인 감정을 시각화한 사진전
박진호는 1990년대에 자신의 벗은 몸을 복사기로 복제하여 실험적인 결과물을 생산한 작가로서 유명하다.
그런데 이번에는 새벽녘의 달과 하늘을 카메라 앵글에 담아서 감상적인 이미지를 전시하였다.
작가가 이번에 인사동에 있는 나우 갤러리에서 전시한 작품들은 달빛과 구름 낀 새벽하늘을 감성적인 느낌이 드는 결과물로 재구성하여 보여주고 있다.
전시 작품마다 외형적으로 컬러가 자극적이고 전체적으로 톤도 어두워서 보는 이들을 감성적으로 동화시키는 표현전략을 구사하고 있다는 것을 알 수 있다.
작가가 관심을 갖고 카메라 앵글에 담은 대상을 구체적으로 살펴보면 구름과 산봉우리 그리고 강변 풍경이다.
그런데 왠지 표현대상과 소재가 낯설게 느껴지지 않고 너무나도 익숙하다.
왜 그런 것 일까?
그것은 작가가 표현대상으로 선택한 소재와 표현방식이 ‘일요 사진가’라고 일컬어지는 아마추어 작가들이 흔히 찍는 탐미적인 사진과 별다른 차이점이 없기 때문이다.
사진은 시각예술이다.
그러므로 작가가 표현하고자하는 주제와 관계없이 외형적으로 보여 지고 느껴지는 것이 무엇보다도 중요하다.
그런데도 아쉽게도 이번에 박진호가 발표한 풍경사진들은 외형적으로 아마추어 작가들의 유미주의적이고 감상적인 작품과의 차별화에 실패하였다.
풍경사진이나 정물사진은 절제된 프레이밍과 세련된 컬러와 톤이 작품의 완성도를 보장하는데 있어서 중요한 요소로 작용한다.
그러므로 주제선택과 더불어서 외형적으로는 그것을 좀 더 염두에 두고 작업을 진행을 했다면 최종 결과물의 완성도가 달라졌을 것이다.
이번에 작가가 발표한 작품들은 누구나 폭넓게 이해할 수 있는 소재와 표현방식을 보여주고 있다. 하지만 자신만의 조형언어를 보여주는 데는 소홀히 했다는 것이 느껴진다.
그래서 여러 가지로 아쉬운 점이 많은 전시가 되었다. 하지만 자신의 감정을 솔직하게 표현하여 결과물을 생산 한 것은 분명하다.
특정한 풍경을 통하여 개인적인 감정을 시각화 전시이다.""".replace('\n', '')

text_splicing(text, max_len) 


[['특정한 풍경을 통하여 개인적인 감정을 시각화한 사진전박진호는 1990년대에 자신의 벗은 몸을 복사기로 복제하여 실험적인 결과물을 생산한 작가로서 유명하다.그런데 이번에는 새벽녘의 달과 하늘을 카메라 앵글에 담아서 감상적인 이미지를 전시하였다.작가가 이번에 인사동에 있는 나우 갤러리에서 전시한 작품들은 달빛과 구름 낀 새벽하늘을 감성적인 느낌이 드는 결과물로 재구성하여 보여주고 있다.전시 작품마다 외형적으로 컬러가 자극적이고 전체적으로 톤도 어두워서 보는 ',
  0],
 ['이들을 감성적으로 동화시키는 표현전략을 구사하고 있다는 것을 알 수 있다.작가가 관심을 갖고 카메라 앵글에 담은 대상을 구체적으로 살펴보면 구름과 산봉우리 그리고 강변 풍경이다.그런데 왠지 표현대상과 소재가 낯설게 느껴지지 않고 너무나도 익숙하다.왜 그런 것 일까?그것은 작가가 표현대상으로 선택한 소재와 표현방식이 ‘일요 사진가’라고 일컬어지는 아마추어 작가들이 흔히 찍는 탐미적인 사진과 별다른 차이점이 없기 때문이다.사진은 시각예술이다.그러므로 작가가 ',
  1],
 ['표현하고자하는 주제와 관계없이 외형적으로 보여 지고 느껴지는 것이 무엇보다도 중요하다.그런데도 아쉽게도 이번에 박진호가 발표한 풍경사진들은 외형적으로 아마추어 작가들의 유미주의적이고 감상적인 작품과의 차별화에 실패하였다.풍경사진이나 정물사진은 절제된 프레이밍과 세련된 컬러와 톤이 작품의 완성도를 보장하는데 있어서 중요한 요소로 작용한다.그러므로 주제선택과 더불어서 외형적으로는 그것을 좀 더 염두에 두고 작업을 진행을 했다면 최종 결과물의 완성도가 달라졌을',
  2],
 [' 것이다.이번에 작가가 발표한 작품들은 누구나 폭넓게 이해할 수 있는 소재와 표현방식을 보여주고 있다. 하지만 자신만의 조형언어를 보여주는 데는 소홀히 했다는 것이 느껴진다.그래서 여러 가지로 아쉬운 점이 많은 전시가 되었다. 하지만 자신의 감정을 솔직하게 표현하여 결과물을 생산 한 것은 분명하다.특정한 풍경을 통하여 개인적인 감정을 시각화 전

In [43]:
import os
content = []
for (path, dir, files) in os.walk(data_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt':
            with open("%s/%s" % (path, filename), encoding="utf-8") as f:
              label = path[path.rindex('/')+1:]
              text = f.read()
              temp_list = text_splicing(text.replace('\n', ' '), max_len)
              if temp_list is not None :
                content.extend(temp_list)

len(content), len(content[0]) # content : [(text, 단락 번호)]

(123265, 2)

In [54]:
content[-2:]

[['않으실거라는 말씀을 전했다. 반백이 되신 어른들이 손자, 손녀에게 한글을 배우라시면서 함께 배우고자 하는 마음이 날 눈물짓게 하는 날들이다. 그리고는 내게 이렇게 오셔서 덕분에 우리가 고려 말을 많이 합니다. 그리고 앞으로도 많이 할 수 있게 되었다 면서......, 고맙다는 인사를 잊지 않는다. 나는 그때마다 영문 모를 죄의식이 든다. 내가 받는 인사가 합당한가? 어쩌면 우리네 집안에 사는 내 나라 땅 안에 사는 우리가 머저리!는 아니었는가? 머리를 조',
  10],
 ['아리게 된다.', 11]]

In [67]:
# label : 0 is True, 1 is False
import random
i, len_content = 0, len(content)
dataset = []
right_li, wrong_li = [], []   # 추가할 list
before_text = content[0][0]
while i < len_content :
  text = content[i][0]
  idx = content[i][1]
  if idx > 0 :    # label : right
    right_li.append( (before_text, text, 0) )   # right

  # idx == 0 and len(li) == 1 인 경우는 없다. file read할 때 넣지 않았다. 
  elif idx == 0 and len(right_li) > 1 :   # label : wrong 
    len_li = len(right_li)
    for t in right_li :
      rd = random.randint(0, len_content-len_li-1)
      wrong_text = content[rd][0] if rd < i-len_li else content[rd+len_li][0]   # random으로 고르기 ( 같은 기사 제외 )
      wrong_li.append((t[0], wrong_text, 1) )
    dataset.extend(right_li)
    dataset.extend(wrong_li)
    right_li, wrong_li = [], []
  before_text = text
  i += 1
len_content, len(dataset)

(123265, 221308)

In [69]:
dataset[-3:]

[('았다. 손님들과 툇마루에 앉아 술을 한 잔 하면 좋을 것 같아 보였다. 종부에게 감사인사를 드린 후, 우리 가족은 이웃한 무송헌 종택으로 이동했다. 무송헌 종택은 무섬에 반쯤 터를 잡고 있는 선성 김씨들의 대종가로 원래 영주시내의 구성산성 옆에 있던 종택이 수십 년 전에 헐리고, 종택이 없는 종가로 남아 위패만 사당에 모시고 있었다. 종가는 무섬 인근 문수면 적동리로 옮겨 살았다. 적동리에서 다시 대구로 이주했던 무송헌 김담 선생의 19대 종손인 김광호씨',
  '장에 모인 사람들 앞에서 발표되었다. 그 연유로 오브나로도바니예라는 단어가 역사적이며, 관습적이고, 가장 일반적이고 광범위한 뜻에서 대중에게 알린다는 뜻을 지니게 되었다. 이에 비해 오푸블리코바니예라는 단어는 현대의 어의적 정밀성이 추가되어 그 어떤 다른 방법이 아닌 신문, 잡지, 단행본 등의 인쇄물로 활자화되어 사람들에게 알린다는 뜻을 지닌다. 왕의 선언만으로도 법률을 확정하고 법률 효력을 발생시켰던 근대 이전과는 다르게, 인쇄술이 발전하고 민주주의가 ',
  1),
 ('가 일가들의 도움으로 2년 전 귀향하면서 새롭게 마련된 종택이다. 종손이 영주에서 공무원 생활을 하다가 대구로 옮겨 20년 가까이 사업을 하다가 2년 전 문중의 도움으로 새롭게 터를 잡았으니, 무송헌 김담 선생의 뜻을 이어받았다는 의미에서 종택의 이름도 그렇게 정했다고 한다. 현판은 안동 퇴계종손이 직접 써준 것이라고 한다. 무송헌(撫松軒) 김담(金淡, 1416년~1464년)선생은 영주출신으로 호는 무송헌이며 조선 세종~문종 때의 명신이다. 이조 판서를 ',
  '세력을 기반으로 하되, 그렇지 않은 분들까지 폭넓은 지지를 받았는데, (지금은) 보수적 지지층의 상당부분이 박 전 대표에게 무게를 실어주는 것으로 본다고 말했다. 박 전 대표의 지지층이 제한돼 있다는 분석으로, 민주당이 향후 경쟁에서 나름의 입지가 있음을 시사한 것으로 보인다. 정동영 출마문제, 민주당 새 평가 받는 데 적절한지 따져봐야 정동영 전 의장의 전주 덕진

In [70]:
from sklearn.model_selection import train_test_split

# Split Train and Validation data
train, test = train_test_split(content[:10000], test_size=0.2, random_state=0, shuffle=True)
train, val = train_test_split(train, test_size=0.2, random_state=0, shuffle=True)
print("train :", len(train), "  val :", len(val), "  test :", len(test))

train : 6400   val : 1600   test : 2000


# 여기까지 ok

In [73]:
model_path = '/content/drive/My Drive/Colab Notebooks/NextLab/nli/result.pt'

In [41]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [45]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
# model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True)

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

+ Korean RoBERTa
+ KorSTS 


In [71]:
from transformers import TFBertForNextSentencePrediction

## BERTNextSentenceModel

In [77]:
from math import log
class BERTNextSentenceModel(pl.LightningModule):
    def __init__(self, config, dataset):
        super().__init__()
        self.config = config
        self.dataset = dataset
        self.slot_labels_type = ["UNK", "PAD", "B", "I"]
        self.pad_token_id = 0

        self.bert_config = BertConfig.from_pretrained(
            self.config.bert_model, num_labels=len(self.slot_labels_type)
        )
        self.model = BertModel.from_pretrained(
            self.config.bert_model, config=self.bert_config
        )
        self.dropout = nn.Dropout(self.config.dropout_rate)
        self.linear = nn.Linear(
            self.bert_config.hidden_size, len(self.slot_labels_type)
        )

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        return self.linear(self.dropout(outputs[0]))

    def training_step(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_label_ids) # slot_labels : labels
        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )

        f1 = self._f1_score(gt_slot_labels, pred_slot_labels)
        acc = self._calculate_accuracy(outputs, slot_label_ids)
        tensorboard_logs = {'train_loss': loss, 'train_f1':f1, 'train_acc':acc}
        return {"loss": loss, "f1": f1, "acc": acc, "log": tensorboard_logs}

    def training_end(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_label_ids) # slot_labels : labels

        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )
        f1 = self._f1_score(gt_slot_labels, pred_slot_labels)
        acc = self._calculate_accuracy(outputs, slot_label_ids)

        tensorboard_logs = {'train_loss': loss, 'train_f1':f1, 'train_acc':acc}
        print("training_end : ", tensorboard_logs)
        return {"loss": loss, "f1": f1, "acc": acc, "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        super().on_train_epoch_end()
        print("training_epoch_end")
      

    def validation_step(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_label_ids)
        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )

        val_f1 = self._f1_score(gt_slot_labels, pred_slot_labels)
        val_acc = self._calculate_accuracy(outputs, slot_label_ids)
        return {"val_loss": loss, "val_f1": val_f1, 'val_acc':val_acc}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        val_f1 = torch.stack([x['val_f1'] for x in outputs]).mean()
        val_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss, 'val_f1':val_f1, 'val_acc':val_acc}
        print('validation_epoch_end : ', tensorboard_logs)
        self.log("validation_epoch_end : tensorboard_logs ", tensorboard_logs)

        return {'val_acc':val_acc, 'val_loss': avg_loss, 'val_f1':val_f1, 'log': tensorboard_logs}
    
    def validation_end(self, outputs):
        val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        val_acc = torch.stack([x["val_acc"] for x in outputs]).mean()
        val_f1 = torch.stack([x["val_f1"] for x in outputs]).mean()
        tensorboard_logs = {
            "val_loss": val_loss,
            "val_f1": val_f1,
            "val_acc" : val_acc
        }
        print("validation_end : ", tensorboard_logs)
        return {'val_acc':val_acc, 'val_loss': val_loss, 'val_f1':val_f1, 'log': tensorboard_logs}
    
    def test_step(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, slot_label_ids = batch
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )
        f1 = self._f1_score(gt_slot_labels, pred_slot_labels)
        acc = self._calculate_accuracy(outputs, slot_label_ids)
        loss = self._calculate_loss(outputs, slot_label_ids)
        return {"test_loss": loss, "test_f1": f1, "test_acc": acc}

    def test_end(self, outputs):
        test_f1 = torch.stack([x["test_f1"] for x in outputs]).mean()
        test_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        test_acc = torch.stack([x["test_acc"] for x in outputs]).mean()
        self.log("test_loss", test_loss)
        self.log("test_acc", test_acc)
        self.log("test_f1", test_f1)
        print('test_end')
        return {"pred_slot_labels" : [x["pred_slot_labels"] for x in outputs], "test_loss": test_loss, "test_f1": test_f1, "test_acc": test_acc}
    
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        f1 = torch.stack([x['test_f1'] for x in outputs]).mean()
        acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        tensorboard_logs = {'test_loss': avg_loss, 'test_f1':f1, 'test_acc':acc}
        print('test_epoch_end : ', tensorboard_logs)
        self.log("test_epoch_end : tensorboard_logs ", tensorboard_logs)
        return {'test_acc':acc, 'test_loss': avg_loss, 'test_f1':f1, 'log': tensorboard_logs}

    def predict_step(self, batch, batch_idx, dataloader_idx=0):   # prediction : forward(), predict_step()
        input_ids, attention_mask, token_type_ids, slot_label_ids = batch    # slot_label은 없음.
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )
        return {'pred_slot_labels':pred_slot_labels, 'gt_slot_labels': gt_slot_labels}


    def configure_optimizers(self):
        return AdamW(self.model.parameters(), lr=2e-5, eps=1e-8)

    def train_dataloader(self):
        return DataLoader(self.dataset["train"], batch_size=self.config.train_batch_size)

    def val_dataloader(self):
        return DataLoader(self.dataset["val"], batch_size=self.config.eval_batch_size)

    def test_dataloader(self):
        return DataLoader(self.dataset["test"], batch_size=self.config.eval_batch_size)

    def pred_dataloader(self, dataset):
        return DataLoader(dataset, batch_size=1)

    def _calculate_loss(self, outputs, labels):
        active_logits = outputs.view(-1, len(self.slot_labels_type))
        active_labels = labels.view(-1)
        loss = F.cross_entropy(active_logits, active_labels)
        return loss
        
    def _calculate_accuracy(self, outputs, labels):
        active_logits = outputs.view(-1, len(self.slot_labels_type))
        active_labels = labels.view(-1)
        accuracy = accuracy_score(active_logits, active_labels)
        return accuracy

    def _f1_score(self, gt_slot_labels, pred_slot_labels):
        return torch.tensor(
            f1_score(gt_slot_labels, pred_slot_labels), dtype=torch.float32
        )

    def _convert_ids_to_labels(self, outputs, slot_labels):
        _, y_hat = torch.max(outputs, dim=2)
        y_hat = y_hat.detach().cpu().numpy()
        slot_label_ids = slot_labels.detach().cpu().numpy()

        slot_label_map = {i: label for i, label in enumerate(self.slot_labels_type)}
        slot_gt_labels = [[] for _ in range(slot_label_ids.shape[0])]
        slot_pred_labels = [[] for _ in range(slot_label_ids.shape[0])]

        for i in range(slot_label_ids.shape[0]):
            for j in range(slot_label_ids.shape[1]):
                if slot_label_ids[i, j] != self.pad_token_id:
                    slot_gt_labels[i].append(slot_label_map[slot_label_ids[i][j]])
                    slot_pred_labels[i].append(slot_label_map[y_hat[i][j]])

        return slot_pred_labels, slot_gt_labels

## Config

In [82]:
# yaml 파일 대신에 객체로 생성
class Config():
  def __init__(self) :
    self.task= 'korean_spacing_20210101'
    self.log_path= data_path+'/logs'
    self.bert_model =  'monologg/kobert'    # 수정해야함!
    self.max_len= 256
    self.train_batch_size= 64
    self.eval_batch_size= 64
    self.dropout_rate= 0.1
    self.gpus= torch.cuda.device_count()
config = Config()

In [80]:
dataset = {}
dataset['train'] = CorpusDataset(train, preprocessor.get_input_features)
dataset['val'] = CorpusDataset(val, preprocessor.get_input_features)
dataset['test'] = CorpusDataset(test, preprocessor.get_input_features)

In [83]:
# model = TFBertForNextSentencePrediction.from_pretrained(model_path)
model2 = BERTNextSentenceModel(config, dataset).load_from_checkpoint(model_path)

TypeError: ignored